# Zellenradschleuse Read/Write Cycle Example

This notebook demonstrates:

1. Discovering Metexon devices (BLE) filtered by the Metexon primary service.
2. Connecting with the typed `ZellenradschleuseClient`.
3. Reading the current system state.
4. Performing a partial manual control update using sentinel-aware helper utilities.
5. Updating the RGB LEDs.
6. Re-reading system state.

> Requires real hardware in range advertising the Metexon service.

If no device is found the notebook will stop early.

In [ ]:
# Discovery step
from metexon import discover_metexon

FOUND = discover_metexon(timeout=5.0)
if not FOUND:
    raise SystemExit("No Metexon devices discovered; cannot continue example.")
FOUND[0]

In [ ]:
# Connect and read
from metexon.zellenradschleuse import ZellenradschleuseClient, sentinels as S
from metexon.zellenradschleuse.update_helpers import partial_manual_control

address = FOUND[0]['address']
client = ZellenradschleuseClient(address)
client.connect()
try:
    print("Device type:", client.device_type())
    ss = client.read_system_state()
    print("Initial state:", ss.to_json())
finally:
    pass  # keep connection open for next cell

In [ ]:
# Perform partial manual control update and LED change
# Use sentinel-based helper to only set feeder_seconds
mc = partial_manual_control(feeder_seconds=1.5)
client.write_manual_control(mc)
print("Issued manual control (feeder 1.5s)")

# Change LEDs
client.write_rgb_led([(255,0,0), (0,255,0)])
print("Updated LEDs")

In [ ]:
# Re-read system state and disconnect
updated = client.read_system_state()
print("Updated state:", updated.to_json())
client.disconnect()
print("Disconnected")